In [1]:
import threading # Importing threading module for using semaphores
import random
import time

# Shared resources
readers_count = 0 # Number of readers reading the resource
readers_count_lock = threading.Semaphore() # Semaphore for readers_count
can_read = threading.Semaphore(0) # Semaphore for readers
can_write = threading.Semaphore(1) # Semaphore for writers
mutex = threading.Semaphore(1) # Semaphore for mutual exclusion


NO_READERS = int(input("Enter the number of readers: ")) # Number of readers
NO_WRITERS = int(input("Enter the number of writers: ")) # Number of writers


# Reader function
def reader(id): 
    global readers_count # Number of readers reading the resource
    mutex.acquire() # Locking the mutex

    readers_count_lock.acquire() # Locking the readers_count_lock
    readers_count += 1 # Incrementing the number of readers
    if readers_count == 1: # If the reader is the first reader
        can_write.acquire() # Locking the can_write semaphore
    readers_count_lock.release() # Releasing the readers_count_lock

    mutex.release() # Releasing the mutex
    can_read.release() # Releasing the can_read semaphore
    resource = open("text.txt", "r") # Opening the resource
    content = resource.read() # Reading the resource
    resource.close() # Closing the resource
    
    print(f"Reader {id} is reading resource: \n{content}") # Printing the content of the resource
    can_read.acquire() 
    
    readers_count_lock.acquire()
    readers_count -= 1 # Decrementing the number of readers
    if readers_count == 0: # If the reader is the last reader
        can_write.release() # Releasing the can_write semaphore
    readers_count_lock.release()

# Writer function
def writer(id):
    time.sleep(random.random()) # Random sleep time
    can_write.acquire() # Locking the can_write semaphore
    mode = input("Enter the mode of writing:\na) append(a)\nb) write(w): ")
    resource = open("text.txt", mode) # Opening the resource
    value = input(f"Enter the text you want to append by writer {id}:")
    value = "\n" + value # Appending a new line before the text
    resource.write(value) # Writing the value to the resource
    resource.close() # Closing the resource
    can_read.release()
    can_write.release()
print("\n")
reader0 = threading.Thread(target=reader, args=(0,))
reader0.start()


reader_list = ["reader0", ] # List of reader threads
writer_list = [] # List of writer threads
for i in range (1, NO_READERS): # Creating the list of reader threads
    var = "reader" + str(i)
    reader_list.append(var)

for i in range (1, NO_WRITERS+1): 
    var = "writer" + str(i)
    writer_list.append(var)

# Starting Function Threads
for i in writer_list:
    i = threading.Thread(target=writer, args=(int(i[-1]),))
    i.start()
    i.join()
    for j in reader_list:
        j = threading.Thread(target=reader, args=(int(j[-1]),))
        j.start()
        j.join()



Reader 0 is reading resource: 
Hello, Reader-writer problem here it is
Helooo
Reader 0 is reading resource: 

Thanks
Reader 1 is reading resource: 

Thanks
